In [ ]:
import numpy as np
import andi_code as andi

In [ ]:
AD = andi.andi_datasets()
AD.avail_models_name

In [ ]:
#we use the save_trajectories feature inside the dataset to quickly generate custom dataset combinations where needed
path = "datasets/trajectories/2d/validset/"
n_samples = int(1e2)
N_save = 2000#500#np.asarray([16000,16000,10000,16000,10000])#500#2000
data = AD.andi_dataset(N = n_samples, tasks = [1, 2],
                           dimensions = [2],
                           load_dataset = False, save_dataset = False, save_trajectories = True, 
                           path_trajectories = path, N_save = N_save)

In [ ]:
from load_andi_dataset import *

T=100
noise_T = T
N_train = int(3e5)
dim = 1
use_increments = True
model = 2
N_save = [16000,16000,10000,16000,1000]

#loading from saved trajectories, allows for only one dataset of trajectories usable for all trajectory lenghts
train_path = "datasets/trajectories/"
train_dataset = SingleModel_dataset_from_saved_trajs(path = train_path, task = 1, dim = dim, N_total = N_train,
                                                  T = T, N_save = N_save[model], 
                                                  use_increments = use_increments, model = model)